In [1]:
# !pip install websockets

In [1]:
import pymssql
from vosk import Model, KaldiRecognizer, SetLogLevel
import wave
import json
import asyncio
import websockets

In [30]:
# cpu approach
model_path = '/mnt/share/audio_call/model_v0/model'
model = Model(model_path)

In [2]:
def connect_sql():

    sql_name = 'voice_ai'
    sql_server = '10.2.4.124'
    sql_login = 'ICECORP\\1c_sql'
    
    with open('sql.pass','r') as file:
        sql_pass = file.read().replace('\n', '')
        file.close()
    
    return pymssql.connect(
        server=sql_server,
        user=sql_login,
        password=sql_pass,
        database=sql_name,
    )

In [54]:
def get_file(source_id):
    
    conn = connect_sql()
    cursor = conn.cursor()
    
    # middle date
    sql_query = "select"
    sql_query += " CAST(AVG(CAST(record_date-0.9 AS FLOAT)) AS DATETIME),"
    sql_query += " CAST(AVG(CAST(record_date+0.1 AS FLOAT)) AS DATETIME)"
    sql_query += " from queue"
    sql_query += " where source_id = '"+source_id+"' and duration>600;"
    cursor.execute(sql_query)
    for row in cursor.fetchall():
        min_time = str(row[0])[:19]
        max_time = str(row[1])[:19]
        print(min_time)
    
    # file name and path from queue
    sql_query = "select top 1 filepath, filename from queue "
    sql_query += "where source_id = '"+source_id+"' "
    sql_query += " and record_date > '" + min_time + "' "
    sql_query += " and record_date < '" + max_time + "' "
    sql_query += " order by record_date, filename;"
    cursor.execute(sql_query)
    for row in cursor.fetchall():
        original_file_path = row[0]
        original_file_name = row[1]
        
    return original_file_path, original_file_name

In [6]:
#original_file_path, original_file_name = get_file('1')
#original_file_path, original_file_name

### MRM records

In [55]:
original_file_path, original_file_name = get_file('2')

2021-08-16 20:26:24


In [56]:
original_file_path, original_file_name

('/mnt/share/audio_master/MSK_MRM/REC_IN_OUT/',
 'a2021-08-16t20:26:55b_c9651424772d_e9647656985f_g1629134813.2604558h-in.wav')

In [57]:
def cpu_approach(model, original_file_path, original_file_name):
    # read file
    wf = wave.open(original_file_path + original_file_name, "rb")
    rec = KaldiRecognizer(model, wf.getframerate())
    phrases_count = 0
    confidences = []
    phrases = []
    while True:

        conf_score = []

        data = wf.readframes(4000)
        if len(data) == 0:
            break

        if rec.AcceptWaveform(data):
            accept = json.loads(rec.Result())
            print('*', accept)
            if accept['text'] != '':

                accept_start = str(accept['result'][0]['start'])
                accept_end = accept['result'][-1:][0]['end']
                accept_text = str(accept['text'])

                for result_rec in accept['result']:
                    conf_score.append(float(result_rec['conf']))
                conf_mid = str(sum(conf_score)/len(conf_score))
                confidences.append(sum(conf_score)/len(conf_score))
                print(accept_start, accept_text)
                phrases.append({
                    'time': accept_start,
                    'conf': conf_mid,
                    'text': accept_text
                })
                phrases_count += 1
    print('end')
    return phrases

In [32]:
phrases_cpu = cpu_approach(model, original_file_path, original_file_name)
phrases_cpu

* {'text': ''}
end


[]

In [62]:
#async def gpu_approach(original_file_path, original_file_name):
server_ip = '10.2.5.212'
uri = 'ws://' + server_ip + ':2700'
phrases_count = 0
confidences = []
phrases = []
async with websockets.connect(uri) as websocket:
    wf = open(original_file_path + original_file_name, "rb")
    while True:
        conf_score = []
        data = wf.read(8000)
        if len(data) == 0:
            break
        await websocket.send(data)
        accept = json.loads(await websocket.recv())
        print('*', accept)
        if len(accept)>1 and accept['text'] != '':

            accept_start = str(accept['result'][0]['start'])
            accept_end = accept['result'][-1:][0]['end']
            accept_text = str(accept['text'])

            for result_rec in accept['result']:
                conf_score.append(float(result_rec['conf']))
            conf_mid = str(sum(conf_score)/len(conf_score))
            confidences.append(sum(conf_score)/len(conf_score))
            print(accept_start, accept_text)
            b.append({
                'time': accept_start,
                'conf': conf_mid,
                'text': accept_text
            })
            #phrases_count += 1
        """if len(accept)>1 and accept['text'] != '':
            pass
            #file_data.append(accept['text'])
            #textfile.write(str(accept['text'])+'\n')
        else:
            [[key, value]] = accept.items()
            if key == 'text' and len(value):
                print('===', value)"""
        
    await websocket.send('{"eof" : 1}')
    print(await websocket.recv())
            
#        return accept
#asyncio.get_event_loop().run_until_complete(
#    gpu_approach(original_file_path, original_file_name)
#)

* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'text': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'text': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': 'алло'}
* {'result': [{'conf': 1.0, 'end': 15.33, 'start': 14.85, 'word': 'алло'}], 'text': 'алло'}
14.85 алло
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': 'вот'}
* {'partial': 'вот'}
* {'partial': 'вот в химках'}
* {'partial': 'вот в химках пока'}
* {'partial': 'вот в химках пока жду'}
* {'partial': 'вот в химках пока жду'}
* {'partial': 'вот в химках пока жду'}
* {'resul

* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': ''}
* {'partial': 'да'}
* {'partial': 'все ненастоящее'}
* {'partial': 'все ненастоящее'}
* {'result': [{'conf': 0.685961, 'end': 94.647246, 'start': 94.44, 'word': 'да'}, {'conf': 0.717483, 'end': 94.89, 'start': 94.659712, 'word': 'все'}, {'conf': 0.514343, 'end': 95.31, 'start': 95.16, 'word': 'они'}, {'conf': 0.514343, 'end': 95.73, 'start': 95.31, 'word': 'настоящие'}], 'text': 'да все они настоящие'}
94.44 да все они настоящие
* {'partial': ''}
* {'partial': ''}
{
  "text" : ""
}


In [63]:
phrases

[{'time': '14.85', 'conf': '1.0', 'text': 'алло'},
 {'time': '18.3',
  'conf': '0.8518686666666667',
  'text': 'вот в химках пока лучшего жду'},
 {'time': '31.47', 'conf': '0.81948125', 'text': 'тут надо быть в'},
 {'time': '35.52',
  'conf': '0.8837776875000001',
  'text': 'давайте я постараюсь до девяти приложу все усилия приятно я думаю в полдевятого для вам подъеду'},
 {'time': '51.916366', 'conf': '0.919291', 'text': 'так я понимаю'},
 {'time': '65.52', 'conf': '1.0', 'text': 'хорошо'},
 {'time': '66.99', 'conf': '1.0', 'text': 'я постараюсь'},
 {'time': '68.43', 'conf': '1.0', 'text': 'где-то в полдевятого подъехать'},
 {'time': '87.63',
  'conf': '0.9814205000000001',
  'text': 'да да до завтра я стараюсь полдевятого к вам подъехать'},
 {'time': '94.44', 'conf': '0.6080325', 'text': 'да все они настоящие'}]